# PRESENTACION TECNICA

## CONTEXTUALIZACION TECNICA

Las respuestas de las empresas a las reclamaciones de los clientes se clasifican en: 
- *Closed with explanation* -> Cerrada con una explicación
- *Closed* -> Cerrada
- *Closed with monetary relief* -> Cerrada con compensación monetaria
- *Closed with non-monetary relief* -> Cerrada con compesación no monetaria

Para simplificar las respuestas éstas pueden ser reclasificadas en dos grupos, reclamaciones que no requieren compensación (*Closed with explanation*, *Closed*) y reclamaciones que si requieren compensación (*Closed with monetary relief*, *Closed with non-monetary relief*).
Dentro del set de datos que se recibió existen columnas que contienen información sobre si la respuesta fue dada a tiempo dentro del marco de tiempo regulatorio (15 días) o no y si el cliente la reclamó o no.
Todas las reclamaciones son importantes, sin embargo aquellas que requieren de algún tipo de acción correctiva por parte de la empresa deben ser prioritarias ya que al contrario de las otras, éstas requieren de modificaciones y/o rectificaciones que reclaman mayor tiempo de trabajo que simplemente una explicación. El 2,5% de las reclamaciones no fueron atendidas a tiempo por las empresas, y de ese porcentaje, el 12% correspondia a reclamaciones que necesitaron algún tip0 de acción correctiva.

Dado este contexto se propuso construir un modelo de *machine learning* capaz de clasificar si las reclamaciones necesitarán o no de acciones correctivas con el fin de darles prioridad y reducir el porcentaje de reclamaciones solucionadas fuera de tiempo.


## METODOLOGIA
Antes de empezar con el todo el proceso de modelado se separó un 20% de los datos (data_test.csv en la carpera de data) para validación que no se utilizaría hasta que el mejor modelo fuese seleccionado.

- El análisis exploratorio de los datos reveló que las columnas *Sub-issue* y *Timely response?* tenían más del 50% de datos faltantes. Excepto la columna *ZIP code*, todas eran varibles objeto. Excepto las columnas *Product*, *Company response*, *Timely response?* y *Consumer disputed*, las variables presentaban una cardinalidad alta. Ya que la mayoría de columnas contenía variables de tipo objeto, el estudio de la asociación entre ellas se realizó mediante la V de Cramér, el cual reveló que las asociaciones por encima de 0.5 se daban entre *Company response* y *Timely response?*, y entre *Product* y *Sub-product*.

- La variable asociada a las respuestas dadas por las empresas se binarizó, ya que el mayor porcentaje eran para el valor *Closed with explanation*, y los demás valores estaban presentes en un porcentaje muy pequeño. La binarización se hizo en respuestas sin compensación y respuestas con compensación. Los valores en la columna representados como *In progress* y *Untimely response* fueron descartados ya que no aportaban información sobre la naturaleza de la respuesta.
Las columnas con las variables *Complaint ID*, *Sub-issue*, *ZIP code*, *Company*,*Timely response?*, *Consumer disputed?* fueron descartadas debido a que no aportaban información, su alta cardinalidad, alta cantidad de datos faltantes o la imposibilidad de realizar feature enginnering con los valores que contenían.


- Los valores faltantes de la variable *Sub-product* se imputaron con el valor "No Subproduct". Después se usó la técnica de *target encoder* que transforma los valores categóricos en numéricos teniendo en cuenta los valores asociados a cada uno de los valores categóricos.

- Los valores de las variables con las fechas de registro de la reclamación y envío a la empresa se desglosaron en día del mes, día de la semana y si era fin de semana o no.

- La variable *Product* dado su baja cardinalidad se codificó mediante la técnica de *OneHotencoding*.

- La variable *State* se recategorizó separada en dos variables para reducir la cardinalidad. Las nuevas variables constituían valores de regiones y divisiones territoriales propuestas por *U.S. Census Bureau* para llevar a cabo estudios estadísticos sociales. Posteriormente estas nuevas variables se codificaron con la técnica *OneHotEncoding*.

- La variable *Issue* que contiene un pequeño texto de la reclamación se codificó como embeddings construidos a partir del modelo Word2Vec. Dentro de cada texto se hizo una media de los embeddings de cada palabra y despues se codificó cada embeddingo con la técnica *OneHotEncoding*.

- Por último, la variable *Company* que contenía los nombre de las empresas presentaba una alta cardinalidad. Para resolver esto y no perder información potencialmente útili se volvió a utilizar la información de la columna *Product* y se reclasificó como actividad empresarial en función del producto implicado en la reclamación.

## MODELADO, RESULTADOS Y METRICAS DE EVALUACION

Con el data set preprocesado se realizó de nuevo una partición del 80% para entrenamiento y un 20% para prueba. 
- Los datos fueron entrenados con modelos de regresión logística, árboles de decisión, máquinas de vectores y un modelo secuencial de red neuronal. Además se utilizaron diferentes ensembles como voting, randomforest, AdaBoost y XGboost. 

- Todos los modelos se entrenaron teniendo en cuenta el desbalance entre la clases, este era en el conjunto de entrenamiento 80% para la respuesta sin compensación y 20% para la respuesta con compensación. Además todos los modelo y ensembles se entrenaron utilizando diferentes combinaciones de hiperparámetros con el fin de explorar lo máximo posible el espacio de soluciones, en algunos casos haciendo uso de la herramienta *GridSearch*.

- Como el objetivo del proyecto era crear un modelo capaz de clasificar si la reclamación necesitaría una compensación o no, y las primeras son las menos representadas se decidió seleccionar el modelo con el f1-score más alto. Es decir aquel modelo en el que la precisión y sensibilidad sobre la clase menos representada estuviesen equilibradas. Además también se utilizó la métrica AUC para seleccionar el modelo con el mayor área bajo la curva, ya que indica el modelo con mayor sensibilidad. Esto hizo disminuir la sensibilidad para detectar la clase más abundante de reclamaciones que no necesitan compensación, sin embargo no es un problema grave que cierto número de reclamaciones que no necesiten compensación sean clasificadas como que si, ya que lo importante es que se detecten un mayor número de reclamaciones que sí las necesitan para poder darle prioridad. 

- El mejor modelo según estos criterios fue un modelo de regresión logística con regularización de Ridge, con una precisión para la clase que necesita compensación de 0.28, sensibilidad de 0.75, f1-score de 0.41 y un AUC de 0.66.

- El prototipo del modelo se construyó con Gradio, siendo el formato de input de valores un archivo csv.


## DISCUSION SOBRE LIMITACIONES Y MEJORAS

- La principal limitación del modelo es que no es capaz de separar con alta precisión y sensibilidad ambas clases. Esto es debido al desbalance que existe entre los casos de respuesta por parte de las empresas. 

- El feature enginnering realizado en el proyecto hasta ahora puede ser mejorado incluyendo variables o recodificando las ya existentes de una forma que ayuden a clasificar mejor la respuesta.

- Una búsqueda má amplia de hiperparámetros, especialmente en las técnicas de ensembles con votación, puede contribuir a que los errores cometidos por los diferentes algoritmos sean compensados y mejore la clasificación.

